In [ ]:
print("策略运行结束")

In [ ]:
from jqdata import *
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re

# 初始化函数 
def initialize(context):
    # 策略基础配置
    set_benchmark('000300.XSHG')  # 沪深300为基准
    set_option('use_real_price', True)  # 真实价格
    #set_option("avoid_future_data", True)  # 防未来函数
    set_slippage(FixedSlippage(0))  # 滑点为0
    set_order_cost(OrderCost(open_tax=0, close_tax=0.001, 
                            open_commission=0.0003, close_commission=0.0003, 
                            close_today_commission=0, min_commission=5), type='fund')  # 交易成本
    log.set_level('order', 'error')  # 过滤订单日志
    
    # 全局变量
    g.stock_num = 100  # 最大持仓数量
    g.hold_list = []  # 当前持仓
    g.final_selection = None  # 存储筛选结果（股票列表）

    # 定时任务：每月1日调仓，每日准备持仓数据
    run_daily(prepare_stock_list, time='9:05')
    run_monthly(adjust_position, 1, time='9:30')  # 每月首交易日调仓
    run_daily(check_limit_up, time='14:00')  # 检查涨停股


# 筛选最终股票池（复用get_final_selection逻辑）
def get_target_stocks(context):
    """获取当月目标股票列表（来自final_selection）"""
    # 日期同步：使用前一交易日作为筛选基准
    yesterday = context.previous_date.strftime('%Y-%m-%d')
    # 获取筛选结果（final_result）
    final_result = get_final_selection(target_date=yesterday)
    # 提取股票代码列表
    return list(final_result['code']) if not final_result.empty else []


# 准备持仓数据（记录当前持仓和昨日涨停股）
def prepare_stock_list(context):
    # 当前持仓列表
    g.hold_list = [pos.security for pos in context.portfolio.positions.values()]
    
    # 昨日涨停股列表（用于调仓时暂留）
    if g.hold_list:
        price_df = get_price(g.hold_list, end_date=context.previous_date, 
                            frequency='daily', fields=['close', 'high_limit'], 
                            count=1, panel=False, fill_paused=False)
        g.high_limit_list = list(price_df[price_df['close'] == price_df['high_limit']]['code'])
    else:
        g.high_limit_list = []


# 调仓逻辑（核心交易执行）
def adjust_position(context):
    
    
    # 新增：仅在1月、4月、7月、10月执行调仓
    current_month = context.current_dt.month
    if current_month not in [1, 4, 7, 10]:
        print(f"当前月份{current_month}月，非调仓月（仅1/4/7/10月调仓），不执行调仓")
        return
    
    
    
    # 1. 获取目标股票列表
    target_list = get_target_stocks(context)
    if not target_list:
        print("无符合条件的目标股票，不进行调仓")
        return
    
    # 2. 过滤不可交易的股票（停牌、涨停、跌停）
    target_list = filter_paused_stock(target_list)
    target_list = filter_limitup_stock(context, target_list)
    target_list = filter_limitdown_stock(context, target_list)
    
    # 3. 限制持仓数量（不超过g.stock_num）
    target_list = target_list[:min(g.stock_num, len(target_list))]
    print(f"本月目标持仓：{len(target_list)}只股票")
    
    g.hold_list = [pos.security for pos in context.portfolio.positions.values()]
    # 4. 卖出操作：不在目标列表且非涨停的股票
    for stock in g.hold_list:
        if stock not in target_list and stock not in g.high_limit_list:
            print(f"卖出：{stock}")
            close_position(context.portfolio.positions[stock])
    g.hold_list = [pos.security for pos in context.portfolio.positions.values()]
    # 5. 买入操作：目标列表中未持仓的股票，平均分配资金
    current_hold_count = len(context.portfolio.positions)
    target_count = len(target_list)
    
    if target_count > current_hold_count:
        # 可用于买入的资金 = 现金 / 需要新增的持仓数量
        invest_cash = context.portfolio.cash / (target_count - current_hold_count)
        for stock in target_list:
            if stock not in g.hold_list:  # 未持仓
                if open_position(stock, invest_cash):  # 按目标金额买入
                    print(f"买入：{stock}，投入资金：{invest_cash:.2f}元")
                    # 达到目标持仓数量则停止
                    if len(context.portfolio.positions) == target_count:
                        break


# 涨停股处理：涨停打开时卖出
def check_limit_up(context):
    if g.high_limit_list:
        for stock in g.high_limit_list:
            # 检查当前价格是否低于涨停价（涨停打开）
            price_df = get_price(stock, end_date=context.current_dt, 
                                frequency='1m', fields=['close', 'high_limit'], 
                                count=1, panel=False, fill_paused=True)
            if price_df.iloc[0]['close'] < price_df.iloc[0]['high_limit']:
                print(f"涨停打开，卖出：{stock}")
                close_position(context.portfolio.positions[stock])


# 工具函数：过滤不可交易股票
def filter_paused_stock(stock_list):
    """过滤停牌股票"""
    current_data = get_current_data()
    return [stock for stock in stock_list if not current_data[stock].paused]

def filter_limitup_stock(context, stock_list):
    """过滤涨停股票（无法买入）"""
    last_prices = history(1, unit='1m', field='close', security_list=stock_list)
    current_data = get_current_data()
    return [stock for stock in stock_list 
            if stock in context.portfolio.positions or  # 已持仓的保留
            last_prices[stock].iloc[-1] < current_data[stock].high_limit]  # 未涨停的可买入

def filter_limitdown_stock(context, stock_list):
    """过滤跌停股票（无法卖出，但此处用于买入过滤）"""
    last_prices = history(1, unit='1m', field='close', security_list=stock_list)
    current_data = get_current_data()
    return [stock for stock in stock_list 
            if stock in context.portfolio.positions or  # 已持仓的保留
            last_prices[stock].iloc[-1] > current_data[stock].low_limit]  # 未跌停的可买入


# 交易模块核心函数
def order_target_value_(security, value):
    """自定义下单函数：按目标市值下单"""
    if value <= 0:
        log.debug(f"下单卖出：{security}")
    else:
        log.debug(f"下单买入：{security}，目标市值：{value:.2f}")
    return order_target_value(security, value)

def open_position(security, value):
    """开仓函数：返回是否成功买入"""
    order = order_target_value_(security, value)
    return order is not None and order.filled > 0

def close_position(position):
    """平仓函数：返回是否成功卖出"""
    order = order_target_value_(position.security, 0)
    return order is not None and order.status == OrderStatus.held and order.filled == order.amount


# 选股模块（复用之前的get_final_selection及相关函数）
# （以下函数与之前保持一致，确保选股逻辑不变）
def standardize_text(text):
    return re.sub(r'[^\w]', '', text)

def filter_stocks(date):
    all_stocks = get_all_securities(types=['stock'], date=date)
    all_symbols = list(all_stocks.index)
    exclude_keywords = ['房地产', '地产', '银行', '金融', '保险', '证券', '货币金融服务']
    
    industry_data = {}
    for i in range(0, len(all_symbols), 1000):
        chunk = all_symbols[i:i+1000]
        industry_data.update(get_industry(chunk, date=date))
    
    valid_stocks = []
    for symbol in all_symbols:
        is_excluded = False
        # 获取行业数据并标准化
        industry_names = []
        for category_data in industry_data.get(symbol, {}).values():
            industry_name = standardize_text(category_data.get('industry_name', ''))
            industry_names.append(industry_name)
        
        # 检查是否存在排除关键词
        for industry_name in industry_names:
            for kw in exclude_keywords:
                if kw in industry_name:
                    is_excluded = True
                    break
            if is_excluded:
                break
                
        if not is_excluded:
            valid_stocks.append(symbol)
    return valid_stocks

def filter_positive_cash_flow(stocks_list, date, years=5):
    date_dt = pd.to_datetime(date)
    latest_year = pd.Timestamp(date).year
    chunks = [stocks_list[i:i+500] for i in range(0, len(stocks_list), 500)]
    all_cash_flow = pd.DataFrame()
    for chunk in chunks:
        try:
            profit_df = get_history_fundamentals(
                chunk,
                [cash_flow.net_operate_cash_flow, cash_flow.pubDate],
                stat_date=f"{latest_year}",
                count=7,  #取7年数据（用于筛选连续5年）
                interval='1y',
                stat_by_year=True
            )
            # 处理日期格式
            profit_df['date'] = date_dt
            profit_df['pubDate'] = pd.to_datetime(profit_df['pubDate'])
            profit_df['statDate'] = pd.to_datetime(profit_df['statDate'])
            # 筛选公告日期在基准日期前的数据
            profit_df = profit_df[profit_df['pubDate'] < profit_df['date']]
            # 按股票代码分组，并在每组内按公告日期降序排序取前5条记录
            result_df = (
                        profit_df
                        .groupby('code', group_keys=False)  # 按code分组但不创建分组键索引
                        .apply(lambda x: x.sort_values('pubDate', ascending=False).head(years))  # 组内倒序取前5
                        .reset_index(drop=True)  # 重置索引
                        )
            all_cash_flow = pd.concat([all_cash_flow, result_df], ignore_index=True)
        except:
            continue
    if all_cash_flow.empty:
        return []
    
    qualified = []
    for code, group in all_cash_flow.groupby('code'):
        if len(group) == years and (group['net_operate_cash_flow'] > 0).all():
            qualified.append(code)
    return qualified

def filter_by_fcff_and_ev(qualified_stocks, date):
    chunks = [qualified_stocks[i:i+500] for i in range(0, len(qualified_stocks), 500)]
    all_results = []
    
    for chunk in chunks:
        try:
            ev_query = query(
                valuation.code,
                (valuation.market_cap*100000000 + balance.total_liability - balance.cash_equivalents).label('enterprise_value')
            ).filter(valuation.code.in_(chunk))
            ev_df = get_fundamentals(ev_query, date=date)
            ev_filtered = ev_df[ev_df['enterprise_value'] > 0].reset_index(drop=True)
            if ev_filtered.empty:
                continue
        except:
            continue
        
        try:
            fcf_data = get_history_fundamentals(
                ev_filtered['code'].tolist(),
                [cash_flow.net_operate_cash_flow, cash_flow.fix_intan_other_asset_acqui_cash],
                watch_date=date, stat_date=None, count=4, interval='1q', stat_by_year=False
            )
            if fcf_data is None or fcf_data.empty:
                continue
        except:
            continue
        
        for code, group in fcf_data.groupby('code'):
            if len(group) != 4:
                continue
            group = group.copy()
            group['fix_intan_other_asset_acqui_cash'] = group['fix_intan_other_asset_acqui_cash'].fillna(0)
            total_op = group['net_operate_cash_flow'].sum()
            total_inv = group['fix_intan_other_asset_acqui_cash'].sum()
            if (total_op - total_inv) > 0:
                ev = ev_filtered[ev_filtered['code'] == code]['enterprise_value'].values[0]
                all_results.append({'code': code, 'enterprise_value': ev, 'free_cash_flow': total_op - total_inv})
    
    if not all_results:
        return pd.DataFrame(columns=['code', 'enterprise_value', 'free_cash_flow', 'fcff_ratio'])
    
    result_df = pd.DataFrame(all_results)
    result_df['fcff_ratio'] = result_df['free_cash_flow'] / result_df['enterprise_value']
    return result_df.sort_values('fcff_ratio', ascending=False).reset_index(drop=True)

def calculate_profit_quality(stocks_df, date):
    stocks_list = stocks_df['code'].tolist()
    chunks = [stocks_list[i:i+500] for i in range(0, len(stocks_list), 500)]
    all_profit_data = []
    
    for chunk in chunks:
        try:
            cash_data = get_history_fundamentals(
                chunk, [cash_flow.net_operate_cash_flow, income.operating_profit],
                watch_date=date, stat_date=None, count=4, interval='1q', stat_by_year=False
            )
            if cash_data is None or cash_data.empty:
                continue
        except:
            continue
        
        annual_data = []
        for code, group in cash_data.groupby('code'):
            if len(group) == 4:
                annual_data.append({
                    'code': code,
                    'annual_net_cash': group['net_operate_cash_flow'].sum(),
                    'annual_operate_profit': group['operating_profit'].sum()
                })
        
        if not annual_data:
            continue
        
        try:
            assets_df = get_fundamentals(
                query(balance.code, balance.total_assets).filter(balance.code.in_([d['code'] for d in annual_data])),
                date=date
            )
            if assets_df.empty:
                continue
        except:
            continue
        
        merged_df = pd.merge(pd.DataFrame(annual_data), assets_df, on='code', how='inner')
        merged_df = merged_df[merged_df['total_assets'] != 0]
        merged_df['profit_quality'] = (merged_df['annual_net_cash'] - merged_df['annual_operate_profit']) / merged_df['total_assets']
        all_profit_data.append(merged_df[['code', 'profit_quality']])
    
    if not all_profit_data:
        return pd.DataFrame(columns=['code', 'profit_quality'])
    
    profit_df = pd.concat(all_profit_data, ignore_index=True)
    profit_df = profit_df.sort_values('profit_quality', ascending=False).reset_index(drop=True)
    return profit_df.head(int(len(profit_df)*0.8)).reset_index(drop=True)

def get_final_selection(target_date=None):
    if target_date is None:
        date = (pd.Timestamp.today() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        date = pd.Timestamp(target_date).strftime('%Y-%m-%d')
    
    print(f"===== 开始筛选（基准日期：{date}） =====")
    
    # 步骤1：行业筛选
    step1 = filter_stocks(date)
    print(f"step1（行业筛选）：{len(step1)}只股票")
    if not step1:
        print("step1无符合条件的股票")
        return pd.DataFrame(columns=['code', 'profit_quality', 'fcff_ratio'])
    
    # 步骤2：连续5年经营现金流为正
    step2 = filter_positive_cash_flow(step1, date, years=5)
    print(f"step2（连续5年现金流为正）：{len(step2)}只股票")
    if not step2:
        print("step2无符合条件的股票")
        return pd.DataFrame(columns=['code', 'profit_quality', 'fcff_ratio'])
    
    # 步骤3：自由现金流和企业价值为正，计算自由现金流率
    step3 = filter_by_fcff_and_ev(step2, date)
    print(f"step3（自由现金流率为正）：{len(step3)}只股票")
    if step3.empty:
        print("step3无符合条件的股票")
        return pd.DataFrame(columns=['code', 'profit_quality', 'fcff_ratio'])
    
    # 步骤4：盈利质量前80%
    step4 = calculate_profit_quality(step3, date)
    print(f"step4（盈利质量前80%）：{len(step4)}只股票")
    if step4.empty:
        print("step4无符合条件的股票")
        return pd.DataFrame(columns=['code', 'profit_quality', 'fcff_ratio'])
    
    # 最终合并
    final = pd.merge(step4, step3[['code', 'fcff_ratio']], on='code', how='inner')
    final_result = final.sort_values('fcff_ratio', ascending=False).head(100).reset_index(drop=True)
    print(f"最终筛选结果：{len(final_result)}只股票")
    return final_result